<br><br><center><b><span style='font-family: "Manjari"; font-size:48px; color:#4285F4'>Google OR-Tools</span></b></center>

---

<h4><b>Import libraries</b></h4>

In [1]:
from __future__ import print_function

# Standard libraries
import os
import random
import math
from itertools import product, permutations, combinations
from collections import Counter, namedtuple, defaultdict

# Common 3rd party libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Optimization libraries and solvers
import ortools

The following examples are from the [OR-Tools tutorials](https://developers.google.com/optimization/introduction/get_started).

---

<br><h4><u>I - Simple LP Example</u></h4>

$$
\begin{align*}
\max_{x} \hspace{4em} 3x_1 + x_2 &\\[5pt]
s.t. \hspace{3em} 0 \le x_1 \le 1 & \\[5pt]
0 \le x_2 \le 2 & \\[5pt]
x_1 + x_2 \le 2 &
\end{align*}
$$

<br>

In [2]:
from ortools.linear_solver import pywraplp

<br> Declare the solver. `GLOP` is Google's own numeric linear solver. Other solvers can be used as well by declaring them. The list of options shows the many options available:

In [3]:
# Create the linear solver with the GLOP backend.
solver = pywraplp.Solver.CreateSolver('GLOP')

In [4]:
solver_info = [print(i) for i in dir(solver) if not i.endswith('__')]

ABNORMAL
AT_LOWER_BOUND
AT_UPPER_BOUND
Add
BASIC
BOP_INTEGER_PROGRAMMING
BoolVar
CBC_MIXED_INTEGER_PROGRAMMING
CLP_LINEAR_PROGRAMMING
CPLEX_LINEAR_PROGRAMMING
CPLEX_MIXED_INTEGER_PROGRAMMING
Clear
ComputeConstraintActivities
ComputeExactConditionNumber
Constraint
CreateSolver
EnableOutput
ExportModelAsLpFormat
ExportModelAsMpsFormat
ExportModelToProto
FEASIBLE
FIXED_VALUE
FREE
FillSolutionResponseProto
GLOP_LINEAR_PROGRAMMING
GLPK_LINEAR_PROGRAMMING
GLPK_MIXED_INTEGER_PROGRAMMING
GUROBI_LINEAR_PROGRAMMING
GUROBI_MIXED_INTEGER_PROGRAMMING
INFEASIBLE
Infinity
IntVar
InterruptSolve
Iterations
LoadModelFromProto
LoadModelFromProtoWithUniqueNamesOrDie
LoadSolutionFromProto
LookupConstraint
LookupVariable
Maximize
Minimize
NOT_SOLVED
NextSolution
NumConstraints
NumVar
NumVariables
OPTIMAL
Objective
RowConstraint
SAT_INTEGER_PROGRAMMING
SCIP_MIXED_INTEGER_PROGRAMMING
SetHint
SetNumThreads
SetSolverSpecificParametersAsString
SetTimeLimit
Solve
SolveWithProto
Sum
SupportsProblemType
SuppressOut

<br> Define the variables:

In [5]:
# Create the variables x_1 and x_2.
x1 = solver.NumVar(0, 1, 'x_1')  # NumVar() creates continuous variables
x2 = solver.NumVar(0, 2, 'x_2')

print('Number of variables =', solver.NumVariables())

Number of variables = 2


<br> Define the constraints:

In [6]:
# Create a linear constraint, 0 <= x_1 + x_2 <= 2.
ct = solver.Constraint(0, 2, 'ct')
ct.SetCoefficient(x1, 1)
ct.SetCoefficient(x2, 1)
# SetCoefficient() gives the coefficient of each variable in the constraint
# e.g. if the constraint was 0 <= 2x_1 - x_2 <= 2, then we would use:
# ct.SetCoefficient(x1, 2)
# ct.SetCoefficient(x2, -1)

print('Number of constraints =', solver.NumConstraints())

Number of constraints = 1


<br> Define the objective:

In [7]:
# Create the objective function, 3 * x_1 + x_2.
objective = solver.Objective()
objective.SetCoefficient(x1, 3)
objective.SetCoefficient(x2, 1)
objective.SetMaximization()  # --> tells the solver this is a maximization problem

<br> Run the solver:

In [8]:
solver.Solve()
print('Solution:')
print('Objective value =', objective.Value())
print('x =', x1.solution_value())
print('y =', x2.solution_value())

Solution:
Objective value = 4.0
x = 1.0
y = 1.0


---

<br><h4><u>II - MIP Example</u></h4>

$$
\begin{align*}
\max \hspace{4em} x + 10y & \\[10pt]
\text{s.t.} \hspace{4em} x + 7 y & \le 17.5 \\[5pt]
x & \le 3.5 \\[5pt]
x & \ge 0 \\[5pt]
y & \ge 0 \\[5pt]
x,\, y \hspace{0.5em} & \text{integers} 
\end{align*}
$$

<br>

In [9]:
# from __future__ import print_function
# from ortools.linear_solver import pywraplp

<br> Different solvers are available for MIP problems. `CBC` is the well-known [COIN-OR Branch and Cut](https://github.com/coin-or/Cbc) solver and `SCIP` is [another](https://www.scipopt.org/) popular solver for MIP problems. <br><i><span style='color:red'>Note:</span> `GLOP` solver is for standard LP problems. If it is chosen, the problem will be solved as a standard LP problem even if the variables are declared as integers.</i>

In [10]:
# Create the mip solver with the CBC or SCIP backend.
solver = pywraplp.Solver.CreateSolver('CBC')
# solver = pywraplp.Solver.CreateSolver('SCIP')
# solver = pywraplp.Solver.CreateSolver('GLOP')

In [11]:
infinity = solver.infinity()  # ? guess this is to instantiate inf as a variable
# x and y are integer non-negative variables.
x = solver.IntVar(0.0, infinity, 'x')  # --> IntVar() specifies int variables
y = solver.IntVar(0.0, infinity, 'y')

print('Number of variables =', solver.NumVariables())

Number of variables = 2


In [12]:
# Add() is used to add constraints

# x + 7 * y <= 17.5.
solver.Add(x + 7 * y <= 17.5)

# x <= 3.5.
solver.Add(x <= 3.5)

print('Number of constraints =', solver.NumConstraints())

Number of constraints = 2


In [13]:
# Maximize x + 10 * y.
solver.Maximize(x + 10 * y)

In [14]:
# Call the solver and assign it to a variable
status = solver.Solve()

<br> With the solver assigned to a variable it can be used to provide more information:

In [15]:
if status == pywraplp.Solver.OPTIMAL:
    print('Solution:')
    print('Objective value =', solver.Objective().Value())
    print('x =', x.solution_value())
    print('y =', y.solution_value())
else:
    print('The problem does not have an optimal solution.')

Solution:
Objective value = 23.0
x = 3.0
y = 2.0


---

<br><h4><u>III - MIP Example with Arrays</u></h4>

$$
\begin{align*}
\max_{x} \hspace{4em} 7x_1 + 8x_2 + 2x_3 + 9x_4 + 6x_5 & \\[10pt]
\text{s.t.} \hspace{4em} 5x_1 + 7x_2 + 9x_3 + 2x_4 + 1x_5 & \le 250 \\[5pt]
18x_1 + 4x_2 - 9x_3 + 10x_4 + 12x_5 & \le 285 \\[5pt]
4x_1 + 7x_2 + 3x_3 + 8x_4 + 5x_5 & \le 211 \\[5pt]
5x_1 + 13x_2 + 16x_3 + 3x_4 - 7x_5 & \le 315 \\[5pt]
x_1, x_2\, \dots x_5 \hspace{0.5em} & \ge 0 \\[5pt]
x_1, x_2\, \dots x_5 \hspace{0.5em} & \text{integers} 
\end{align*}
$$

<br>

In [16]:
# Create the mip solver with the CBC or SCIP backend.
solver = pywraplp.Solver.CreateSolver('CBC')
# solver = pywraplp.Solver.CreateSolver('mip_program_with_arrays', 'SCIP')

<br> A `dict` is convenient for storing array values. This `data_model` format is used throughout and is very helpful for storing the model inputs.

In [17]:
def create_data_model():
    data = dict()
    data['constraint_coeffs'] = [[5, 7, 9, 2, 1], 
                                 [18, 4, -9, 10, 12], 
                                 [4, 7, 3, 8, 5], 
                                 [5, 13, 16, 3, -7] 
                                ]
    data['upper_bounds'] = [250, 285, 211, 315]
    data['obj_coeffs'] = [7, 8, 2, 9, 6]
    data['num_vars'] = 5
    data['num_constraints'] = 4
    return data

data = create_data_model()

In [18]:
infinity = solver.infinity()

x = dict()
for j in range(data['num_vars']):
    x[j] = solver.IntVar(0, infinity, f'x[{j}]')
    
print('Number of variables =', solver.NumVariables())

Number of variables = 5


In [19]:
x

{0: x[0], 1: x[1], 2: x[2], 3: x[3], 4: x[4]}

In [20]:
for i in range(data['num_constraints']):
    constraint = solver.RowConstraint(0, data['upper_bounds'][i], '')
    for j in range(data['num_vars']):
        constraint.SetCoefficient(x[j], data['constraint_coeffs'][i][j])

print('Number of constraints =', solver.NumConstraints())

# for i in range(data['num_constraints']):
#     constraint_expr = [data['constraint_coeffs'][i][j] * x[j] for j in range(data['num_vars'])]
#     solver.Add(sum(constraint_expr) <= data['bounds'][i])

Number of constraints = 4


In [21]:
objective = solver.Objective()

for j in range(data['num_vars']):
    objective.SetCoefficient(x[j], data['obj_coeffs'][j])
    
objective.SetMaximization()

# obj_expr = [data['obj_coeffs'][j] * x[j] for j in range(data['num_vars'])]
# solver.Maximize(solver.Sum(obj_expr))

In [22]:
time_begin = solver.wall_time()
status = solver.Solve()
time_end = solver.wall_time()
solution_time = time_end - time_begin

In [23]:
if status == pywraplp.Solver.OPTIMAL:
    print('Objective value =', solver.Objective().Value())
    for j in range(data['num_vars']):
        print(x[j].name(), ' = ', x[j].solution_value())
    print()
    print(f'Problem solved in {solution_time} milliseconds')
    print(f'Problem solved in {solver.iterations()} iterations')
    print(f'Problem solved in {solver.nodes()} branch-and-bound nodes')

else:
    print('The problem does not have an optimal solution.')

Objective value = 260.0
x[0]  =  10.0
x[1]  =  16.0
x[2]  =  4.0
x[3]  =  4.0
x[4]  =  3.0

Problem solved in 17 milliseconds
Problem solved in 315 iterations
Problem solved in 13 branch-and-bound nodes


---

<br> <i>The following is not in the tutorials</i>. This is just another way of solving the above problem but in a more compact, Python-based approach:

$$
\begin{align*}
\max_{x} \hspace{4em} 7x_1 + 8x_2 + 2x_3 + 9x_4 + 6x_5 & \\[10pt]
\text{s.t.} \hspace{4em} 5x_1 + 7x_2 + 9x_3 + 2x_4 + 1x_5 & \le 250 \\[5pt]
18x_1 + 4x_2 - 9x_3 + 10x_4 + 12x_5 & \le 285 \\[5pt]
4x_1 + 7x_2 + 3x_3 + 8x_4 + 5x_5 & \le 211 \\[5pt]
5x_1 + 13x_2 + 16x_3 + 3x_4 - 7x_5 & \le 315 \\[5pt]
x_1, x_2\, \dots x_5 \hspace{0.5em} & \ge 0 \\[5pt]
x_1, x_2\, \dots x_5 \hspace{0.5em} & \text{integers} 
\end{align*}
$$

<br>

In [24]:
# Create the mip solver with the CBC or SCIP backend.
solver = pywraplp.Solver.CreateSolver('CBC')
# solver = pywraplp.Solver.CreateSolver('mip_program_with_arrays', 'SCIP')

<br> Replace lists in the `data` dictionary with Numpy arrays:

In [25]:
def data_model():
    data = dict()
    data['constraint_coeffs'] = np.array([[5, 7, 9, 2, 1], 
                                          [18, 4, -9, 10, 12], 
                                          [4, 7, 3, 8, 5], 
                                          [5, 13, 16, 3, -7] 
                                         ])
    data['upper_bounds'] = np.array([250, 285, 211, 315])
    data['obj_coeffs'] = np.array([7, 8, 2, 9, 6])
    data['num_vars'] = data['constraint_coeffs'].shape[1]  # no. of columns in constraints
    data['num_constraints'] = data['constraint_coeffs'].shape[0]  # no. of rows
    return data

data = data_model()

<br> Using a dictionary as in the original tutorial to store $x_i$ variables seems unnecessary, so use an array instead. The nested `for` loops can be replaced with a dot product which is not only more succinct, but also true to the canonical form of optimization and quicker to process.

In [26]:
# define infinity value
infinity = solver.infinity()
# define constraint relationships
x = np.array([solver.IntVar(0, infinity, f'x[{i}]') for i in range(data.get('num_vars')) ])
constraint_eqns = np.dot(data.get('constraint_coeffs'), x)
for i in range(constraint_eqns.shape[0]):
    solver.Add(constraint_eqns[i] <= data.get('upper_bounds')[i])

print('Number of variables =', solver.NumVariables())
print('Number of constraints =', solver.NumConstraints())

Number of variables = 5
Number of constraints = 4


<br> The above applies for defining the objective as well:

In [27]:
objective_fn = np.dot(data.get('obj_coeffs'), x)
solver.Maximize(objective_fn)

<br> The rest is identical to the previous approach:

In [28]:
time_begin = solver.wall_time()
status = solver.Solve()
time_end = solver.wall_time()
solution_time = time_end - time_begin

In [29]:
if status == pywraplp.Solver.OPTIMAL:
    print('Objective value =', solver.Objective().Value())
    for i in range(data['num_vars']):
        print(x[i].name(), ' = ', x[i].solution_value())
    print()
    print(f'Problem solved in {solution_time} milliseconds')
    print(f'Problem solved in {solver.iterations()} iterations')
    print(f'Problem solved in {solver.nodes()} branch-and-bound nodes')

else:
    print('The problem does not have an optimal solution.')

Objective value = 260.0
x[0]  =  10.0
x[1]  =  16.0
x[2]  =  4.0
x[3]  =  4.0
x[4]  =  3.0

Problem solved in 24 milliseconds
Problem solved in 344 iterations
Problem solved in 19 branch-and-bound nodes


---

<br><h4><u>IV - Constraint Programming Example</u></h4>

Constraint Programming (CP) is the main paradigm used by OR-Tools for solving vehicle routing problems. This example is used as a simpe intro to CP before continuing on to VRP.

In [30]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from ortools.constraint_solver import pywrapcp

This first simple problem is a quick introduction. The goal is to find a <i>feasible</i> (not necessarily optimal) solution to the following problem:
- $x, \, y,$ and $z$ are three variables that can take on values $0, \, 1,$ or $2$.
- One constraint: $x \ne y$

Declare the CP solver:

In [31]:
solver = pywrapcp.Solver("simple_CP_example")

<br> Create variables:

In [32]:
# CP solver can only take integers
x = solver.IntVar(0, 2, 'x')
y = solver.IntVar(0, 2, 'y')
z = solver.IntVar(0, 2, 'z')

<br> Create the constraint:

In [33]:
solver.Add(x != y)

<br> Create a solver and solve the model:

In [34]:
def print_solution(solver, x, y, z):
    count = 0

    while solver.NextSolution():
        count += 1
        print("x =", x.Value(), "y =", y.Value(), "z =", z.Value())
    print("\nNumber of solutions found:", count)

<br> The CP solver uses a <i>decision builder</i> to assign variables and pass options to the solver. The `Phase()` method is one way (of many) to create a decision builder. More info [here](https://developers.google.com/optimization/cp/original_cp_solver#decision-builder).

In [35]:
db = solver.Phase([x, y, z], solver.CHOOSE_FIRST_UNBOUND, solver.ASSIGN_MIN_VALUE)

In [36]:
solver.Solve(db)
print_solution(solver, x, y, z)

x = 0 y = 1 z = 0
x = 0 y = 1 z = 1
x = 0 y = 1 z = 2
x = 0 y = 2 z = 0
x = 0 y = 2 z = 1
x = 0 y = 2 z = 2
x = 1 y = 0 z = 0
x = 1 y = 0 z = 1
x = 1 y = 0 z = 2
x = 1 y = 2 z = 0
x = 1 y = 2 z = 1
x = 1 y = 2 z = 2
x = 2 y = 0 z = 0
x = 2 y = 0 z = 1
x = 2 y = 0 z = 2
x = 2 y = 1 z = 0
x = 2 y = 1 z = 1
x = 2 y = 1 z = 2

Number of solutions found: 18


---

<br> <center><h3><b>Vehicle Routing and TSP</b></h3></center>

<br><h4><u>V - TSP Example</u></h4>

In this problem, it is required to visit 13 different cities in the US:

0. New York 
1. Los Angeles 
2. Chicago 
3. Minneapolis 
4. Denver 
5. Dallas 
6. Seattle 
7. Boston 
8. San Francisco 
9. St. Louis 
10. Houston 
11. Phoenix 
12. Salt Lake City

New York is the starting and end point (designated as the 'depot') and the number of vehicles is $\it 1$ for a TSP problem. See the [tutorial page](https://developers.google.com/optimization/routing/tsp) for a map and more info.

In [37]:
# import the solver and functions
from __future__ import print_function
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

<br> As usual, the first step is to create the data model. Because this is a TSP, the most important input is the distance matrix which represents the graph structure with locations as nodes and distances as edges.

In [38]:
def tsp_data_model():
    data = dict()
    data['distance_matrix'] = np.array([[0, 2451, 713, 1018, 1631, 1374, 2408, 213, 2571, 875, 1420, 2145, 1972],
                                        [2451, 0, 1745, 1524, 831, 1240, 959, 2596, 403, 1589, 1374, 357, 579],
                                        [713, 1745, 0, 355, 920, 803, 1737, 851, 1858, 262, 940, 1453, 1260],
                                        [1018, 1524, 355, 0, 700, 862, 1395, 1123, 1584, 466, 1056, 1280, 987],
                                        [1631, 831, 920, 700, 0, 663, 1021, 1769, 949, 796, 879, 586, 371],
                                        [1374, 1240, 803, 862, 663, 0, 1681, 1551, 1765, 547, 225, 887, 999],
                                        [2408, 959, 1737, 1395, 1021, 1681, 0, 2493, 678, 1724, 1891, 1114, 701],
                                        [213, 2596, 851, 1123, 1769, 1551, 2493, 0, 2699, 1038, 1605, 2300, 2099],
                                        [2571, 403, 1858, 1584, 949, 1765, 678, 2699, 0, 1744, 1645, 653, 600],
                                        [875, 1589, 262, 466, 796, 547, 1724, 1038, 1744, 0, 679, 1272, 1162],
                                        [1420, 1374, 940, 1056, 879, 225, 1891, 1605, 1645, 679, 0, 1017, 1200],
                                        [2145, 357, 1453, 1280, 586, 887, 1114, 2300, 653, 1272, 1017, 0, 504],
                                        [1972, 579, 1260, 987, 371, 999, 701, 2099, 600, 1162, 1200, 504, 0],
                                       ])
    data['num_vehicles'] = 1
    data['depot'] = 0
    return data

In [39]:
data = tsp_data_model()

<br> The manager below makes it easier to work with the nodes without having to access the solvers internal indexing structure.

In [40]:
manager = pywrapcp.RoutingIndexManager(data.get('distance_matrix').shape[0],  # number of nodes
                                       data.get('num_vehicles'),  # number of vehicles
                                       data.get('depot'))  # starting and end point

routing = pywrapcp.RoutingModel(manager)  # creates the model

<br> The following just shows the incredible number of methods and parameters available for `RoutingModel()`.

In [41]:
routing_methods = [print(i) for i in dir(routing) if not i.endswith('__')]

ADDED_TYPE_REMOVED_FROM_VEHICLE
ActiveVar
ActiveVehicleVar
AddAtSolutionCallback
AddConstantDimension
AddConstantDimensionWithSlack
AddDimension
AddDimensionWithVehicleCapacity
AddDimensionWithVehicleTransitAndCapacity
AddDimensionWithVehicleTransits
AddDisjunction
AddHardTypeIncompatibility
AddIntervalToAssignment
AddLocalSearchFilter
AddLocalSearchOperator
AddMatrixDimension
AddPickupAndDelivery
AddPickupAndDeliverySets
AddRequiredTypeAlternativesWhenAddingType
AddRequiredTypeAlternativesWhenRemovingType
AddSameVehicleRequiredTypeAlternatives
AddSearchMonitor
AddSoftSameVehicleConstraint
AddTemporalTypeIncompatibility
AddToAssignment
AddVariableMaximizedByFinalizer
AddVariableMinimizedByFinalizer
AddVariableTargetToFinalizer
AddVectorDimension
AddWeightedVariableMinimizedByFinalizer
ApplyLocks
ApplyLocksToAllVehicles
ArcIsMoreConstrainedThanArc
AreEmptyRouteCostsConsideredForVehicle
AssignmentToRoutes
CheckLimit
CloseModel
CloseModelWithParameters
CloseVisitTypes
CompactAndCheckAssig

<br> The following function creates the callback (which returns the distance between any two nodes) and registers it with the solver as `transit_callback_index`.

In [42]:
def distance_callback(from_index, to_index):
    """Returns the distance between the two nodes."""
    # Convert from routing variable Index to distance matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data.get('distance_matrix')[from_node, to_node]

transit_callback_index = routing.RegisterTransitCallback(distance_callback)

<br> This next line tells the solver that the cost is simply equal to the distance traveled. However, this cost can take different values functions and can also be assigned per vehicle using `SetArcCostEvaluatorOfVehicle()`.

In [43]:
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

<br> Set the solution strategy and defaults for the search:

In [44]:
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

<br> This function prints out the best route and objective value. Note that "best" here does not always mean optimal, since the default parameters may limit the amount of searching if the solution space is very large.

In [45]:
def print_solution(manager, routing, solution):
    """Prints solution on console."""
    print('Objective: {} miles'.format(solution.ObjectiveValue()))
    index = routing.Start(0)
    plan_output = 'Route for vehicle 0:\n'
    route_distance = 0
    while not routing.IsEnd(index):
        plan_output += ' {} ->'.format(manager.IndexToNode(index))
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
    plan_output += ' {}\n'.format(manager.IndexToNode(index))
    print(plan_output)
    plan_output += 'Route distance: {}miles\n'.format(route_distance)

<br> Finally, start solving:

In [46]:
solution = routing.SolveWithParameters(search_parameters)
if solution:
    print_solution(manager, routing, solution)

Objective: 7293 miles
Route for vehicle 0:
 0 -> 7 -> 2 -> 3 -> 4 -> 12 -> 6 -> 8 -> 1 -> 11 -> 10 -> 5 -> 9 -> 0



<br> Instead of printing, results can be saved to a list:

In [47]:
def get_routes(solution, routing, manager):
    """Get vehicle routes from a solution and store them in an array."""
    # Get vehicle routes and store them in a two dimensional array whose
    # i,j entry is the jth location visited by vehicle i along its route.
    routes = []
    for route_nbr in range(routing.vehicles()):
        index = routing.Start(route_nbr)
        route = [manager.IndexToNode(index)]
    while not routing.IsEnd(index):
        index = solution.Value(routing.NextVar(index))
        route.append(manager.IndexToNode(index))
    routes.append(route)
    return routes

In [48]:
routes = get_routes(solution, routing, manager)
# Display the routes.
for i, route in enumerate(routes):
    print('Route ' + str(i) + ':', ' -> '.join([str(i) for i in route]))

Route 0: 0 -> 7 -> 2 -> 3 -> 4 -> 12 -> 6 -> 8 -> 1 -> 11 -> 10 -> 5 -> 9 -> 0


---

<br><h4><u>VI - VRP Example</u></h4>

There are different ways to define the "optimal routes" for multiple vehicles for VRP problems. In this problem, the optimal routes are the ones that minimize the length of the longest single route among all vehicles. <br>In this example, the goal is to find the optimal routes for $4$ vehicles serving $16$ locations in a city arranged on identical rectangular blocks. More info [here](https://developers.google.com/optimization/routing/vrp).

In [49]:
# import the solver and functions

# from __future__ import print_function
# from ortools.constraint_solver import routing_enums_pb2
# from ortools.constraint_solver import pywrapcp

In [50]:
def vrp_data_model():
    """Stores the data for the problem."""
    data = dict()
    data['distance_matrix'] = np.array([
        [
            0, 548, 776, 696, 582, 274, 502, 194, 308, 194, 536, 502, 388, 354,
            468, 776, 662
        ],
        [
            548, 0, 684, 308, 194, 502, 730, 354, 696, 742, 1084, 594, 480, 674,
            1016, 868, 1210
        ],
        [
            776, 684, 0, 992, 878, 502, 274, 810, 468, 742, 400, 1278, 1164,
            1130, 788, 1552, 754
        ],
        [
            696, 308, 992, 0, 114, 650, 878, 502, 844, 890, 1232, 514, 628, 822,
            1164, 560, 1358
        ],
        [
            582, 194, 878, 114, 0, 536, 764, 388, 730, 776, 1118, 400, 514, 708,
            1050, 674, 1244
        ],
        [
            274, 502, 502, 650, 536, 0, 228, 308, 194, 240, 582, 776, 662, 628,
            514, 1050, 708
        ],
        [
            502, 730, 274, 878, 764, 228, 0, 536, 194, 468, 354, 1004, 890, 856,
            514, 1278, 480
        ],
        [
            194, 354, 810, 502, 388, 308, 536, 0, 342, 388, 730, 468, 354, 320,
            662, 742, 856
        ],
        [
            308, 696, 468, 844, 730, 194, 194, 342, 0, 274, 388, 810, 696, 662,
            320, 1084, 514
        ],
        [
            194, 742, 742, 890, 776, 240, 468, 388, 274, 0, 342, 536, 422, 388,
            274, 810, 468
        ],
        [
            536, 1084, 400, 1232, 1118, 582, 354, 730, 388, 342, 0, 878, 764,
            730, 388, 1152, 354
        ],
        [
            502, 594, 1278, 514, 400, 776, 1004, 468, 810, 536, 878, 0, 114,
            308, 650, 274, 844
        ],
        [
            388, 480, 1164, 628, 514, 662, 890, 354, 696, 422, 764, 114, 0, 194,
            536, 388, 730
        ],
        [
            354, 674, 1130, 822, 708, 628, 856, 320, 662, 388, 730, 308, 194, 0,
            342, 422, 536
        ],
        [
            468, 1016, 788, 1164, 1050, 514, 514, 662, 320, 274, 388, 650, 536,
            342, 0, 764, 194
        ],
        [
            776, 868, 1552, 560, 674, 1050, 1278, 742, 1084, 810, 1152, 274,
            388, 422, 764, 0, 798
        ],
        [
            662, 1210, 754, 1358, 1244, 708, 480, 856, 514, 468, 354, 844, 730,
            536, 194, 798, 0
        ],
    ])
    data['num_vehicles'] = 4
    data['depot'] = 0
    return data

In [51]:
data = vrp_data_model()

<br> Set up the manager, callback and search parameters. This is identical to the preceding TSP example.

In [52]:
# Managers
manager = pywrapcp.RoutingIndexManager(data.get('distance_matrix').shape[0],  # number of nodes
                                       data.get('num_vehicles'),  # number of vehicles
                                       data.get('depot'))  # starting and end point

routing = pywrapcp.RoutingModel(manager)

def distance_callback(from_index, to_index):
    """Returns the distance between the two nodes."""
    # Convert from routing variable Index to distance matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data.get('distance_matrix')[from_node, to_node]

# Define cost of each arc.
transit_callback_index = routing.RegisterTransitCallback(distance_callback)
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

# Setting first solution heuristic.
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

<br> <span style='color:red'><b>IMPORTANT
    <br></b></span><b>Dimensions:</b> For VRP problems, <i>dimensions</i> are required. The dimensions are used to tell the solver how to accumulate the cost(s) over the distance traveled by each vehicle. Properly defining dimensions is critical to finding a good solution to the VRP problem and its variations.


In [53]:
dimension_name = 'Distance'  # important: make sure to use unique names for each dimension

routing.AddDimension(transit_callback_index, 
                     0,  # no slack 
                     3000,  # vehicle maximum travel distance 
                     True,  # start cumul to zero 
                     dimension_name)

distance_dimension = routing.GetDimensionOrDie(dimension_name)

distance_dimension.SetGlobalSpanCostCoefficient(100)  # use a large value for this (like 100)

<br> Same print function as above, slightly modified for multiple vehicles:

In [54]:
def print_vrp_solution(data, manager, routing, solution):
    
    max_route_distance = 0
    total_all_routes = 0
    
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        
        while not routing.IsEnd(index):
            plan_output += ' {} -> '.format(manager.IndexToNode(index))
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(previous_index, index, vehicle_id)
        
        plan_output += '{}\n'.format(manager.IndexToNode(index))
        plan_output += 'Distance of the route: {}m\n'.format(route_distance)
        print(plan_output)
        max_route_distance = max(route_distance, max_route_distance)
        total_all_routes += route_distance
    
    print('Maximum of the route distances: {}m'.format(max_route_distance))
    print('Total distance of all routes: {}m'.format(total_all_routes))

<br> Solve!

In [55]:
solution = routing.SolveWithParameters(search_parameters)
if solution:
    print_vrp_solution(data, manager, routing, solution)

Route for vehicle 0:
 0 ->  8 ->  6 ->  2 ->  5 -> 0
Distance of the route: 1552m

Route for vehicle 1:
 0 ->  7 ->  1 ->  4 ->  3 -> 0
Distance of the route: 1552m

Route for vehicle 2:
 0 ->  9 ->  10 ->  16 ->  14 -> 0
Distance of the route: 1552m

Route for vehicle 3:
 0 ->  12 ->  11 ->  15 ->  13 -> 0
Distance of the route: 1552m

Maximum of the route distances: 1552m
Total distance of all routes: 6208m


---

<br><h4><u>VII - CVRP Example</u></h4>

This example is the same as above but adds capacity constraints. In this case, the locations and distances are the same as before but now each location has a demand (deliveries to either pick up or drop off) and each vehicle has a maximum capacity it can carry.

In [56]:
# from __future__ import print_function
# from ortools.constraint_solver import routing_enums_pb2
# from ortools.constraint_solver import pywrapcp

The data model is initialized as in the VRP model but there are two additional data values: <i>demand</i> at each location and <i>capacities</i>. The capacity is the same for all vehicles in this example but different capacities can be used.

In [57]:
def cvrp_data_model():
    """Stores the data for the problem."""
    data = dict()
    data['distance_matrix'] = np.array([
        [
            0, 548, 776, 696, 582, 274, 502, 194, 308, 194, 536, 502, 388, 354,
            468, 776, 662
        ],
        [
            548, 0, 684, 308, 194, 502, 730, 354, 696, 742, 1084, 594, 480, 674,
            1016, 868, 1210
        ],
        [
            776, 684, 0, 992, 878, 502, 274, 810, 468, 742, 400, 1278, 1164,
            1130, 788, 1552, 754
        ],
        [
            696, 308, 992, 0, 114, 650, 878, 502, 844, 890, 1232, 514, 628, 822,
            1164, 560, 1358
        ],
        [
            582, 194, 878, 114, 0, 536, 764, 388, 730, 776, 1118, 400, 514, 708,
            1050, 674, 1244
        ],
        [
            274, 502, 502, 650, 536, 0, 228, 308, 194, 240, 582, 776, 662, 628,
            514, 1050, 708
        ],
        [
            502, 730, 274, 878, 764, 228, 0, 536, 194, 468, 354, 1004, 890, 856,
            514, 1278, 480
        ],
        [
            194, 354, 810, 502, 388, 308, 536, 0, 342, 388, 730, 468, 354, 320,
            662, 742, 856
        ],
        [
            308, 696, 468, 844, 730, 194, 194, 342, 0, 274, 388, 810, 696, 662,
            320, 1084, 514
        ],
        [
            194, 742, 742, 890, 776, 240, 468, 388, 274, 0, 342, 536, 422, 388,
            274, 810, 468
        ],
        [
            536, 1084, 400, 1232, 1118, 582, 354, 730, 388, 342, 0, 878, 764,
            730, 388, 1152, 354
        ],
        [
            502, 594, 1278, 514, 400, 776, 1004, 468, 810, 536, 878, 0, 114,
            308, 650, 274, 844
        ],
        [
            388, 480, 1164, 628, 514, 662, 890, 354, 696, 422, 764, 114, 0, 194,
            536, 388, 730
        ],
        [
            354, 674, 1130, 822, 708, 628, 856, 320, 662, 388, 730, 308, 194, 0,
            342, 422, 536
        ],
        [
            468, 1016, 788, 1164, 1050, 514, 514, 662, 320, 274, 388, 650, 536,
            342, 0, 764, 194
        ],
        [
            776, 868, 1552, 560, 674, 1050, 1278, 742, 1084, 810, 1152, 274,
            388, 422, 764, 0, 798
        ],
        [
            662, 1210, 754, 1358, 1244, 708, 480, 856, 514, 468, 354, 844, 730,
            536, 194, 798, 0
        ],
    ])
    data['demands'] = [0, 1, 1, 2, 4, 2, 4, 8, 8, 1, 2, 1, 2, 4, 4, 8, 8]
    data['vehicle_capacities'] = [15, 20, 15, 12]
    data['num_vehicles'] = 4
    data['depot'] = 0
    return data

In [58]:
data = cvrp_data_model()

<br> Again, set up the manager and the distance call back as in the VRP problem. The arc cost is also based on travel distance, as in the VRP example.

In [59]:
# Manager
manager = pywrapcp.RoutingIndexManager(data.get('distance_matrix').shape[0],  # number of nodes
                                       data.get('num_vehicles'),  # number of vehicles
                                       data.get('depot'))  # starting and end point
# Model
routing = pywrapcp.RoutingModel(manager)

In [60]:
def distance_callback(from_index, to_index):
    """Returns the distance between the two nodes."""
    # Convert from routing variable Index to distance matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data.get('distance_matrix')[from_node, to_node]

# Register transit callback and define cost of each arc.
transit_callback_index = routing.RegisterTransitCallback(distance_callback)
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

<br> Another important modification for capacity constraints is adding an additional callback for the demand.

In [61]:
# Add Capacity constraint.
def demand_callback(from_index):
    """Returns the demand of the node."""
    # Convert from routing variable Index to demands NodeIndex.
    from_node = manager.IndexToNode(from_index)
    return data.get('demands')[from_node]

# Register capacity callback
demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)

In this CVRP model, instead of using `addDimension()` as before (which can still be used here if all vehicles have the same capacity) the method `addDimensionWithVehicleCapacity()` is used.

In [62]:
routing.AddDimensionWithVehicleCapacity(demand_callback_index, 
                                        0, # null capacity slack
                                        data.get('vehicle_capacities'), # vehicle maximum capacities
                                        True, # start cumul to zero
                                        'Capacity')

True

The rest of the model proceeds as before. Set the solution parameters and define a print function.

In [63]:
# Setting first solution heuristic.
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

In [64]:
def print_cvrp_solution(data, manager, routing, solution):
    total_distance = 0
    total_load = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data['demands'][node_index]
            plan_output += ' {0} Load({1}) -> '.format(node_index, route_load)
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        plan_output += ' {0} Load({1})\n'.format(manager.IndexToNode(index),
                                                 route_load)
        plan_output += 'Distance of the route: {}m\n'.format(route_distance)
        plan_output += 'Load of the route: {}\n'.format(route_load)
        print(plan_output)
        total_distance += route_distance
        total_load += route_load
    print('Total distance of all routes: {}m'.format(total_distance))
    print('Total load of all routes: {}'.format(total_load))

<br> Solve!

In [65]:
solution = routing.SolveWithParameters(search_parameters)
if solution:
    print_cvrp_solution(data, manager, routing, solution)

Route for vehicle 0:
 0 Load(0) ->  1 Load(1) ->  4 Load(5) ->  3 Load(7) ->  15 Load(15) ->  0 Load(15)
Distance of the route: 2192m
Load of the route: 15

Route for vehicle 1:
 0 Load(0) ->  9 Load(1) ->  14 Load(5) ->  16 Load(13) ->  10 Load(15) ->  2 Load(16) ->  6 Load(20) ->  0 Load(20)
Distance of the route: 2192m
Load of the route: 20

Route for vehicle 2:
 0 Load(0) ->  7 Load(8) ->  13 Load(12) ->  12 Load(14) ->  11 Load(15) ->  0 Load(15)
Distance of the route: 1324m
Load of the route: 15

Route for vehicle 3:
 0 Load(0) ->  5 Load(2) ->  8 Load(10) ->  0 Load(10)
Distance of the route: 776m
Load of the route: 10

Total distance of all routes: 6484m
Total load of all routes: 60


---

<br><h4><u>VIII - VRPTW Example</u></h4>

The previous example showed a capacity-constrained problem. In this example, the constraints are time-based instead of capacity-based. This requires some changes to the input data to solve:
- The distance matrix is replaced with a time matrix defining the travel time between two points.
- The time windows for each delivery/visit must be added.

In [66]:
def vrptw_data_model():
    """Stores the data for the problem."""
    data = {}
    data['time_matrix'] = np.array([[0, 6, 9, 8, 7, 3, 6, 2, 3, 2, 6, 6, 4, 4, 5, 9, 7],
                                    [6, 0, 8, 3, 2, 6, 8, 4, 8, 8, 13, 7, 5, 8, 12, 10, 14],
                                    [9, 8, 0, 11, 10, 6, 3, 9, 5, 8, 4, 15, 14, 13, 9, 18, 9],
                                    [8, 3, 11, 0, 1, 7, 10, 6, 10, 10, 14, 6, 7, 9, 14, 6, 16],
                                    [7, 2, 10, 1, 0, 6, 9, 4, 8, 9, 13, 4, 6, 8, 12, 8, 14],
                                    [3, 6, 6, 7, 6, 0, 2, 3, 2, 2, 7, 9, 7, 7, 6, 12, 8],
                                    [6, 8, 3, 10, 9, 2, 0, 6, 2, 5, 4, 12, 10, 10, 6, 15, 5],
                                    [2, 4, 9, 6, 4, 3, 6, 0, 4, 4, 8, 5, 4, 3, 7, 8, 10],
                                    [3, 8, 5, 10, 8, 2, 2, 4, 0, 3, 4, 9, 8, 7, 3, 13, 6],
                                    [2, 8, 8, 10, 9, 2, 5, 4, 3, 0, 4, 6, 5, 4, 3, 9, 5],
                                    [6, 13, 4, 14, 13, 7, 4, 8, 4, 4, 0, 10, 9, 8, 4, 13, 4],
                                    [6, 7, 15, 6, 4, 9, 12, 5, 9, 6, 10, 0, 1, 3, 7, 3, 10],
                                    [4, 5, 14, 7, 6, 7, 10, 4, 8, 5, 9, 1, 0, 2, 6, 4, 8],
                                    [4, 8, 13, 9, 8, 7, 10, 3, 7, 4, 8, 3, 2, 0, 4, 5, 6],
                                    [5, 12, 9, 14, 12, 6, 6, 7, 3, 3, 4, 7, 6, 4, 0, 9, 2],
                                    [9, 10, 18, 6, 8, 12, 15, 8, 13, 9, 13, 3, 4, 5, 9, 0, 9],
                                    [7, 14, 9, 16, 14, 8, 5, 10, 6, 5, 4, 10, 8, 6, 2, 9, 0] ])
    # each tuple represents the time window (beginning, end) for each location
    # starting from loc 0 (depot) to loc 16
    data['time_windows'] = [(0, 5), 
                            (7, 12), 
                            (10, 15), 
                            (16, 18), 
                            (10, 13), 
                            (0, 5), 
                            (5, 10), 
                            (0, 4), 
                            (5, 10), 
                            (0, 3), 
                            (10, 16), 
                            (10, 15), 
                            (0, 5), 
                            (5, 10), 
                            (7, 8), 
                            (10, 15), 
                            (11, 15) ]  # cannot be np array (?)
    data['num_vehicles'] = 4
    data['depot'] = 0
    return data

In [67]:
data = vrptw_data_model()

<br> The next step is similar to before: create the routing index manager and model.

In [68]:
# Create the routing index manager.
manager = pywrapcp.RoutingIndexManager(data.get('time_matrix').shape[0],  # use the time matrix
                                       data.get('num_vehicles'), 
                                       data.get('depot'))

# Create Routing Model.
routing = pywrapcp.RoutingModel(manager)

<br> For the VRPTW model, the transit callback is based on time instead of distance:

In [69]:
def time_callback(from_index, to_index):
    """Returns the travel time between the two nodes."""
    # Convert from routing variable Index to time matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data.get('time_matrix')[from_node, to_node]

transit_callback_index = routing.RegisterTransitCallback(time_callback)

# Define cost of each arc.
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

<br> In the next block, the time window constraints are added. Here the slack is set to 30 min to allow any required waiting time and the maximum capacity is also set to 30, meaning a vehicle's total time on the route cannot exceed 30 min. Also notice the change in `force_cumul_start` from `True` to `False`.

In [70]:
time = 'Time'

routing.AddDimension(transit_callback_index, 
                     30,  # allow waiting time
                     30,  # maximum time per vehicle
                     False,  # Don't force start cumul to zero.
                     time)

time_dimension = routing.GetDimensionOrDie(time)

# Add time window constraints for each location except depot.
for location_idx, time_window in enumerate(data.get('time_windows')):
    if location_idx == 0:
        continue
    index = manager.NodeToIndex(location_idx)
    time_dimension.CumulVar(index).SetRange(time_window[0], time_window[1])
    
# Add time window constraints for each vehicle start node.
for vehicle_id in range(data.get('num_vehicles')):
    index = routing.Start(vehicle_id)
    time_dimension.CumulVar(index).SetRange(data.get('time_windows')[0][0],
                                            data.get('time_windows')[0][1])

<br> These next lines appear to be necessary to actually solve for feasible routes within the given constraints. They are not explained at all in the original example on the OR-Tools website.

In [71]:
# Instantiate route start and end times to produce feasible times.
for i in range(data['num_vehicles']):
    routing.AddVariableMinimizedByFinalizer(
        time_dimension.CumulVar(routing.Start(i)))
    routing.AddVariableMinimizedByFinalizer(
        time_dimension.CumulVar(routing.End(i)))

<br> The first solution heuristic is just like previous examples:

In [72]:
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

<br> Again, an updated print function for this model:

In [73]:
def print_vrptw_solution(data, manager, routing, solution):
    """Prints solution on console."""
    time_dimension = routing.GetDimensionOrDie('Time')
    total_time = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        while not routing.IsEnd(index):
            time_var = time_dimension.CumulVar(index)
            plan_output += '{0} Time({1},{2}) -> '.format(
                manager.IndexToNode(index), solution.Min(time_var),
                solution.Max(time_var))
            index = solution.Value(routing.NextVar(index))
        time_var = time_dimension.CumulVar(index)
        plan_output += '{0} Time({1},{2})\n'.format(manager.IndexToNode(index),
                                                    solution.Min(time_var),
                                                    solution.Max(time_var))
        plan_output += 'Time of the route: {}min\n'.format(
            solution.Min(time_var))
        print(plan_output)
        total_time += solution.Min(time_var)
    print('Total time of all routes: {}min'.format(total_time))

<br> Solve!

In [74]:
solution = routing.SolveWithParameters(search_parameters)

In [75]:
# store and print the results
if solution:
    print_vrptw_solution(data, manager, routing, solution)

Route for vehicle 0:
0 Time(0,0) -> 9 Time(2,3) -> 14 Time(7,8) -> 16 Time(11,11) -> 0 Time(18,18)
Time of the route: 18min

Route for vehicle 1:
0 Time(0,0) -> 7 Time(2,4) -> 1 Time(7,11) -> 4 Time(10,13) -> 3 Time(16,16) -> 0 Time(24,24)
Time of the route: 24min

Route for vehicle 2:
0 Time(0,0) -> 12 Time(4,4) -> 13 Time(6,6) -> 15 Time(11,11) -> 11 Time(14,14) -> 0 Time(20,20)
Time of the route: 20min

Route for vehicle 3:
0 Time(0,0) -> 5 Time(3,3) -> 8 Time(5,5) -> 6 Time(7,7) -> 2 Time(10,10) -> 10 Time(14,14) -> 0 Time(20,20)
Time of the route: 20min

Total time of all routes: 82min


---

<br><h4><u>IX - <i>CVRPTW</i> Example</u></h4>

The previous examples separately showed a capacitated vehicle routing problem (CVRP) and vehicle routing with time windows (VRPTW) problem. The tutorials on the OR-Tools website do not offer a CVRPTW example where the routing problem is constrained by both time windows and capacity. This example attempts to build on the previous two examples to solve a problem of this type.

<br> The first step is to update the data model. The updated data model for the CVRPTW problem combines the time matrix of the VRPTW example with the capacity constraints of the CVRP model. The time windows here are modified from the original model to make it easier to find feasible solutions.

In [76]:
def cvrp_tw_data_model():
    """Stores the data for the problem."""
    data = {}
    data['time_matrix'] = np.array([[0, 6, 9, 8, 7, 3, 6, 2, 3, 2, 6, 6, 4, 4, 5, 9, 7],
                                    [6, 0, 8, 3, 2, 6, 8, 4, 8, 8, 13, 7, 5, 8, 12, 10, 14],
                                    [9, 8, 0, 11, 10, 6, 3, 9, 5, 8, 4, 15, 14, 13, 9, 18, 9],
                                    [8, 3, 11, 0, 1, 7, 10, 6, 10, 10, 14, 6, 7, 9, 14, 6, 16],
                                    [7, 2, 10, 1, 0, 6, 9, 4, 8, 9, 13, 4, 6, 8, 12, 8, 14],
                                    [3, 6, 6, 7, 6, 0, 2, 3, 2, 2, 7, 9, 7, 7, 6, 12, 8],
                                    [6, 8, 3, 10, 9, 2, 0, 6, 2, 5, 4, 12, 10, 10, 6, 15, 5],
                                    [2, 4, 9, 6, 4, 3, 6, 0, 4, 4, 8, 5, 4, 3, 7, 8, 10],
                                    [3, 8, 5, 10, 8, 2, 2, 4, 0, 3, 4, 9, 8, 7, 3, 13, 6],
                                    [2, 8, 8, 10, 9, 2, 5, 4, 3, 0, 4, 6, 5, 4, 3, 9, 5],
                                    [6, 13, 4, 14, 13, 7, 4, 8, 4, 4, 0, 10, 9, 8, 4, 13, 4],
                                    [6, 7, 15, 6, 4, 9, 12, 5, 9, 6, 10, 0, 1, 3, 7, 3, 10],
                                    [4, 5, 14, 7, 6, 7, 10, 4, 8, 5, 9, 1, 0, 2, 6, 4, 8],
                                    [4, 8, 13, 9, 8, 7, 10, 3, 7, 4, 8, 3, 2, 0, 4, 5, 6],
                                    [5, 12, 9, 14, 12, 6, 6, 7, 3, 3, 4, 7, 6, 4, 0, 9, 2],
                                    [9, 10, 18, 6, 8, 12, 15, 8, 13, 9, 13, 3, 4, 5, 9, 0, 9],
                                    [7, 14, 9, 16, 14, 8, 5, 10, 6, 5, 4, 10, 8, 6, 2, 9, 0] ])
    # each tuple represents the time window (beginning, end) for each location
    # starting from loc 0 (depot) to loc 16
    data['time_windows'] = [(0, 60), 
                            (6, 14), 
                            (8, 16), 
                            (15, 22), 
                            (9, 15), 
                            (0, 8), 
                            (5, 20), 
                            (0, 12), 
                            (5, 10), 
                            (0, 6), 
                            (15, 25), 
                            (12, 20), 
                            (0, 7), 
                            (5, 15), 
                            (6, 12), 
                            (10, 20), 
                            (15, 28) ]  # cannot be np array (?)
    
    data['demands'] = [0, 1, 1, 2, 4, 2, 4, 8, 8, 1, 2, 1, 2, 4, 4, 8, 8]
    
    data['vehicle_capacities'] = [15, 20, 15, 12, 15, 20, 15]
    
    data['num_vehicles'] = 7
    
    data['depot'] = 0
    
    return data

In [77]:
data = cvrp_tw_data_model()

<br> Create the manager and model as before. For this example, the solution is driven again by a time matrix because of the time window constraints.

In [78]:
# Create the routing index manager.
manager = pywrapcp.RoutingIndexManager(data.get('time_matrix').shape[0],  # use the time matrix
                                       data.get('num_vehicles'), 
                                       data.get('depot'))

# Create Routing Model.
routing = pywrapcp.RoutingModel(manager)

<br> The time callback is also defined the same as in the VRPTW problem:

In [79]:
def time_callback(from_index, to_index):
    """Returns the travel time between the two nodes."""
    # Convert from routing variable Index to time matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data.get('time_matrix')[from_node, to_node]

transit_callback_index = routing.RegisterTransitCallback(time_callback)

# Define cost of each arc.
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

<br> Again, define the time constraints. In this example, the waiting time and maximum time are increased to make it easier to find feasible solutions:

In [80]:
time = 'Time'

routing.AddDimension(transit_callback_index, 
                     45,  # allow waiting time
                     45,  # maximum time per vehicle
                     False,  # Don't force start cumul to zero.
                     time)

time_dimension = routing.GetDimensionOrDie(time)

# Add time window constraints for each location except depot.
for location_idx, time_window in enumerate(data.get('time_windows')):
    if location_idx == 0:
        continue
    index = manager.NodeToIndex(location_idx)
    time_dimension.CumulVar(index).SetRange(time_window[0], time_window[1])
    
# Add time window constraints for each vehicle start node.
for vehicle_id in range(data.get('num_vehicles')):
    index = routing.Start(vehicle_id)
    time_dimension.CumulVar(index).SetRange(data.get('time_windows')[0][0],
                                            data.get('time_windows')[0][1])

<br> Next, add the demand callback as in the CVRP problem and define the capacity dimension.

In [81]:
# Add Capacity constraint.
def demand_callback(from_index):
    """Returns the demand of the node."""
    # Convert from routing variable Index to demands NodeIndex.
    from_node = manager.IndexToNode(from_index)
    return data.get('demands')[from_node]

# Register capacity callback
demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)

In [82]:
routing.AddDimensionWithVehicleCapacity(demand_callback_index, 
                                        0, # null capacity slack
                                        data.get('vehicle_capacities'), # vehicle maximum capacities
                                        True, # start cumul to zero
                                        'Capacity')

True

<br> As before, instantiate route start and end times to get feasible routes that satisfy the time constraints:

In [83]:
for i in range(data['num_vehicles']):
    routing.AddVariableMinimizedByFinalizer(
        time_dimension.CumulVar(routing.Start(i)))
    routing.AddVariableMinimizedByFinalizer(
        time_dimension.CumulVar(routing.End(i)))

<br> Initialize the search heuristic:

In [84]:
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
# search_parameters.local_search_metaheuristic = (
#     routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
# search_parameters.time_limit.seconds = 30
# search_parameters.log_search = True

<br> Define the print function. The two print functions below are simply copies of the functions used in the CRVP and VRPTW models above, with the CVRP function just slightly modified to remove the distance output (since this model uses travel time which is already output by the first function). Of course, the two print functions can be merged into one but here they are kept separate to make things easy.

In [85]:
def print_tw_solution(data, manager, routing, solution):
    """Prints solution on console."""
    time_dimension = routing.GetDimensionOrDie('Time')
    total_time = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        while not routing.IsEnd(index):
            time_var = time_dimension.CumulVar(index)
            plan_output += '{0} Time({1},{2}) -> '.format(
                manager.IndexToNode(index), solution.Min(time_var),
                solution.Max(time_var))
            index = solution.Value(routing.NextVar(index))
        time_var = time_dimension.CumulVar(index)
        plan_output += '{0} Time({1},{2})\n'.format(manager.IndexToNode(index),
                                                    solution.Min(time_var),
                                                    solution.Max(time_var))
        plan_output += 'Time of the route: {}min\n'.format(
            solution.Min(time_var))
        print(plan_output)
        total_time += solution.Min(time_var)
    print('Total time of all routes: {}min'.format(total_time))

    
def print_load_solution(data, manager, routing, solution):
    total_load = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data['demands'][node_index]
            plan_output += ' {0} Load({1}) -> '.format(node_index, route_load)
            previous_index = index
            index = solution.Value(routing.NextVar(index))
        plan_output += ' {0} Load({1})\n'.format(manager.IndexToNode(index),
                                                 route_load)
        plan_output += 'Load of the route: {}\n'.format(route_load)
        print(plan_output)
        total_load += route_load
    print('Total load of all routes: {}'.format(total_load))

<br> Finally, solve the problem and inspect the output!

In [86]:
solution = routing.SolveWithParameters(search_parameters)

In [87]:
# store and print the results
if solution:
    print_tw_solution(data, manager, routing, solution)
    print('\n'+'='*100,'\n')
    print_load_solution(data, manager, routing, solution)

Route for vehicle 0:
0 Time(0,0) -> 0 Time(0,0)
Time of the route: 0min

Route for vehicle 1:
0 Time(0,0) -> 8 Time(5,5) -> 5 Time(7,7) -> 6 Time(9,9) -> 2 Time(12,12) -> 10 Time(16,16) -> 0 Time(22,22)
Time of the route: 22min

Route for vehicle 2:
0 Time(0,0) -> 7 Time(2,8) -> 1 Time(6,12) -> 4 Time(9,14) -> 3 Time(15,15) -> 0 Time(23,23)
Time of the route: 23min

Route for vehicle 3:
0 Time(0,0) -> 0 Time(0,0)
Time of the route: 0min

Route for vehicle 4:
0 Time(0,0) -> 9 Time(2,6) -> 14 Time(6,12) -> 16 Time(15,15) -> 0 Time(22,22)
Time of the route: 22min

Route for vehicle 5:
0 Time(0,0) -> 13 Time(5,5) -> 12 Time(7,7) -> 15 Time(11,11) -> 11 Time(14,14) -> 0 Time(20,20)
Time of the route: 20min

Route for vehicle 6:
0 Time(0,0) -> 0 Time(0,0)
Time of the route: 0min

Total time of all routes: 87min


Route for vehicle 0:
 0 Load(0) ->  0 Load(0)
Load of the route: 0

Route for vehicle 1:
 0 Load(0) ->  8 Load(8) ->  5 Load(10) ->  6 Load(14) ->  2 Load(15) ->  10 Load(17) ->  0 

In [88]:
routing.status()

1

---

<br><h4><u>X - <span style='color:purple'>Simulation of Final Problem</span> </u></h4>

This final example simulates the "Growing Pains" case we are solving. It is a CPRVTW problem as above but adds some additional constraints. The most significant is that there is are upper limits on both drive time and total time. The objective of the Growing Pains problem is to minimize total distance per year. To tackle this, the objective function is set as the distance with an upper bound set by translating the max drive time into a max distance.

This leads to three callbacks and three dimensions for applying constraints:
- Distance with upper bound max distance
- Total time with an upper bound (e.g. total duty hours per driver)
- Capacity with upper bound vehicle capacity

With these in place, the problem proceeds as before.

<br> The first step is to update the data model. The updated data model for the CVRPTW problem combines the time matrix of the VRPTW example with the capacity constraints of the CVRP model. The time windows here are modified from the original model to make it easier to find feasible solutions.

In [89]:
def cvrp_tw_data_model():
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = np.array([
        [
            0, 548, 776, 696, 582, 274, 502, 194, 308, 194, 536, 502, 388, 354,
            468, 776, 662
        ],
        [
            548, 0, 684, 308, 194, 502, 730, 354, 696, 742, 1084, 594, 480, 674,
            1016, 868, 1210
        ],
        [
            776, 684, 0, 992, 878, 502, 274, 810, 468, 742, 400, 1278, 1164,
            1130, 788, 1552, 754
        ],
        [
            696, 308, 992, 0, 114, 650, 878, 502, 844, 890, 1232, 514, 628, 822,
            1164, 560, 1358
        ],
        [
            582, 194, 878, 114, 0, 536, 764, 388, 730, 776, 1118, 400, 514, 708,
            1050, 674, 1244
        ],
        [
            274, 502, 502, 650, 536, 0, 228, 308, 194, 240, 582, 776, 662, 628,
            514, 1050, 708
        ],
        [
            502, 730, 274, 878, 764, 228, 0, 536, 194, 468, 354, 1004, 890, 856,
            514, 1278, 480
        ],
        [
            194, 354, 810, 502, 388, 308, 536, 0, 342, 388, 730, 468, 354, 320,
            662, 742, 856
        ],
        [
            308, 696, 468, 844, 730, 194, 194, 342, 0, 274, 388, 810, 696, 662,
            320, 1084, 514
        ],
        [
            194, 742, 742, 890, 776, 240, 468, 388, 274, 0, 342, 536, 422, 388,
            274, 810, 468
        ],
        [
            536, 1084, 400, 1232, 1118, 582, 354, 730, 388, 342, 0, 878, 764,
            730, 388, 1152, 354
        ],
        [
            502, 594, 1278, 514, 400, 776, 1004, 468, 810, 536, 878, 0, 114,
            308, 650, 274, 844
        ],
        [
            388, 480, 1164, 628, 514, 662, 890, 354, 696, 422, 764, 114, 0, 194,
            536, 388, 730
        ],
        [
            354, 674, 1130, 822, 708, 628, 856, 320, 662, 388, 730, 308, 194, 0,
            342, 422, 536
        ],
        [
            468, 1016, 788, 1164, 1050, 514, 514, 662, 320, 274, 388, 650, 536,
            342, 0, 764, 194
        ],
        [
            776, 868, 1552, 560, 674, 1050, 1278, 742, 1084, 810, 1152, 274,
            388, 422, 764, 0, 798
        ],
        [
            662, 1210, 754, 1358, 1244, 708, 480, 856, 514, 468, 354, 844, 730,
            536, 194, 798, 0
        ],
    ])
    data['time_matrix'] = np.array([[0, 6, 9, 8, 7, 3, 6, 2, 3, 2, 6, 6, 4, 4, 5, 9, 7],
                                    [6, 0, 8, 3, 2, 6, 8, 4, 8, 8, 13, 7, 5, 8, 12, 10, 14],
                                    [9, 8, 0, 11, 10, 6, 3, 9, 5, 8, 4, 15, 14, 13, 9, 18, 9],
                                    [8, 3, 11, 0, 1, 7, 10, 6, 10, 10, 14, 6, 7, 9, 14, 6, 16],
                                    [7, 2, 10, 1, 0, 6, 9, 4, 8, 9, 13, 4, 6, 8, 12, 8, 14],
                                    [3, 6, 6, 7, 6, 0, 2, 3, 2, 2, 7, 9, 7, 7, 6, 12, 8],
                                    [6, 8, 3, 10, 9, 2, 0, 6, 2, 5, 4, 12, 10, 10, 6, 15, 5],
                                    [2, 4, 9, 6, 4, 3, 6, 0, 4, 4, 8, 5, 4, 3, 7, 8, 10],
                                    [3, 8, 5, 10, 8, 2, 2, 4, 0, 3, 4, 9, 8, 7, 3, 13, 6],
                                    [2, 8, 8, 10, 9, 2, 5, 4, 3, 0, 4, 6, 5, 4, 3, 9, 5],
                                    [6, 13, 4, 14, 13, 7, 4, 8, 4, 4, 0, 10, 9, 8, 4, 13, 4],
                                    [6, 7, 15, 6, 4, 9, 12, 5, 9, 6, 10, 0, 1, 3, 7, 3, 10],
                                    [4, 5, 14, 7, 6, 7, 10, 4, 8, 5, 9, 1, 0, 2, 6, 4, 8],
                                    [4, 8, 13, 9, 8, 7, 10, 3, 7, 4, 8, 3, 2, 0, 4, 5, 6],
                                    [5, 12, 9, 14, 12, 6, 6, 7, 3, 3, 4, 7, 6, 4, 0, 9, 2],
                                    [9, 10, 18, 6, 8, 12, 15, 8, 13, 9, 13, 3, 4, 5, 9, 0, 9],
                                    [7, 14, 9, 16, 14, 8, 5, 10, 6, 5, 4, 10, 8, 6, 2, 9, 0] ])
    # each tuple represents the time window (beginning, end) for each location
    # starting from loc 0 (depot) to loc 16
    data['time_windows'] = [(0, 60), 
                            (6, 14), 
                            (8, 16), 
                            (15, 22), 
                            (9, 15), 
                            (0, 8), 
                            (5, 20), 
                            (0, 12), 
                            (5, 10), 
                            (0, 6), 
                            (15, 25), 
                            (12, 20), 
                            (0, 7), 
                            (5, 15), 
                            (6, 12), 
                            (10, 20), 
                            (15, 28) ]  # cannot be np array (?)
    
    data['wait_times'] = [0, 2, 0, 1, 2, 2, 1, 1, 2, 2, 0, 2, 2, 0, 2, 1, 1]  # cannot be np array (?)
    
    data['demands'] = [0, 1, 1, 2, 4, 2, 4, 8, 8, 1, 2, 1, 2, 4, 4, 8, 8]
    
    data['vehicle_capacities'] = [25, 20, 15, 25, 15, 20, 20, 15]
    
    data['num_vehicles'] = 8
    
    data['depot'] = 0
    
    return data

In [90]:
data = cvrp_tw_data_model()

<br> Create the manager and model as before. For this example, the solution is driven again by a time matrix because of the time window constraints.

In [91]:
# Create the routing index manager.
manager = pywrapcp.RoutingIndexManager(data.get('distance_matrix').shape[0],  # use the time matrix
                                       data.get('num_vehicles'), 
                                       data.get('depot'))

# Create Routing Model.
routing = pywrapcp.RoutingModel(manager)

<br> The time callback is also defined the same as in the VRPTW problem:

In [92]:
def distance_callback(from_index, to_index):
    """Returns the distance between the two nodes."""
    # Convert from routing variable Index to distance matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data.get('distance_matrix')[from_node, to_node]

# Register transit callback and define cost of each arc.
distance_callback_index = routing.RegisterTransitCallback(distance_callback)

# Objective function
routing.SetArcCostEvaluatorOfAllVehicles(distance_callback_index)

In [93]:
def time_callback(from_index, to_index):
    """Returns the travel time between the two nodes."""
    # Convert from routing variable Index to time matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data.get('time_matrix')[from_node, to_node] + data.get('wait_times')[from_node]

time_callback_index = routing.RegisterTransitCallback(time_callback)

# Define cost of each arc.
# routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

<br> Again, define the time constraints. In this example, the waiting time and maximum time are increased to make it easier to find feasible solutions:

In [94]:
time = 'Time'

routing.AddDimension(time_callback_index, 
                     0,  # allow waiting time
                     45,  # maximum time per vehicle
                     False,  # Don't force start cumul to zero.
                     time)

time_dimension = routing.GetDimensionOrDie(time)

# Add time window constraints for each location except depot.
for location_idx, time_window in enumerate(data.get('time_windows')):
    if location_idx == 0:
        continue
    index = manager.NodeToIndex(location_idx)
    time_dimension.CumulVar(index).SetRange(time_window[0], time_window[1])
    

dist = 'Distance'

routing.AddDimension(distance_callback_index, 
                     0,  # allow slack
                     1600,  # maximum dist per vehicle
                     True,  # Don't force start cumul to zero.
                     dist)

distance_dimension = routing.GetDimensionOrDie(dist)

<br> Next, add the demand callback as in the CVRP problem and define the capacity dimension.

In [95]:
# Add Capacity constraint.
def demand_callback(from_index):
    """Returns the demand of the node."""
    # Convert from routing variable Index to demands NodeIndex.
    from_node = manager.IndexToNode(from_index)
    return data.get('demands')[from_node]

# Register capacity callback
demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)

In [96]:
routing.AddDimensionWithVehicleCapacity(demand_callback_index, 
                                        0, # null capacity slack
                                        data.get('vehicle_capacities'), # vehicle maximum capacities
                                        True, # start cumul to zero
                                        'Capacity')

True

<br> As before, instantiate route start and end times to get feasible routes that satisfy the time constraints:

In [97]:
for i in range(data['num_vehicles']):
    routing.AddVariableMinimizedByFinalizer(
        time_dimension.CumulVar(routing.Start(i)))
    routing.AddVariableMinimizedByFinalizer(
        time_dimension.CumulVar(routing.End(i)))

<br> Initialize the search heuristic:

In [98]:
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
# search_parameters.local_search_metaheuristic = (
#     routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
search_parameters.time_limit.seconds = 15
search_parameters.log_search = True

<br> Define the print function. The two print functions below are simply copies of the functions used in the CRVP and VRPTW models above, with the CVRP function just slightly modified to remove the distance output (since this model uses travel time which is already output by the first function). Of course, the two print functions can be merged into one but here they are kept separate to make things easy.

In [99]:
def print_tw_solution(data, manager, routing, solution):
    """Prints solution on console."""
    time_dimension = routing.GetDimensionOrDie('Time')
    dist_dimension = routing.GetDimensionOrDie('Distance')
    total_time = 0
    total_dist = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        while not routing.IsEnd(index):
            time_var = time_dimension.CumulVar(index)
            dist_var = dist_dimension.CumulVar(index)
            plan_output += '{0} Time({1},{2}) Dist({3})-> '.format(
                manager.IndexToNode(index), solution.Min(time_var),
                solution.Max(time_var), solution.Value(dist_var))
            index = solution.Value(routing.NextVar(index))
        time_var = time_dimension.CumulVar(index)
        plan_output += '{0} Time({1},{2}) Dist({3})\n'.format(manager.IndexToNode(index),
                                                    solution.Min(time_var),
                                                    solution.Max(time_var), 
                                                    solution.Value(dist_var))
        plan_output += 'Time of the route: {}min\n'.format(
            solution.Min(time_var))
        plan_output += 'Route distance: {}m\n'.format(
            solution.Value(dist_var))
        print(plan_output)
        total_time += solution.Min(time_var)
        total_dist += solution.Value(dist_var)
    print(f'Total time of all routes: {total_time}min', 
          f'\nTotal distance of all routes: {total_dist}m')

    
def print_load_solution(data, manager, routing, solution):
    total_load = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data['demands'][node_index]
            plan_output += ' {0} Load({1}) -> '.format(node_index, route_load)
            previous_index = index
            index = solution.Value(routing.NextVar(index))
        plan_output += ' {0} Load({1})\n'.format(manager.IndexToNode(index),
                                                 route_load)
        plan_output += 'Load of the route: {}\n'.format(route_load)
        print(plan_output)
        total_load += route_load
    print('Total load of all routes: {}'.format(total_load))

<br> Finally, solve the problem and inspect the output!

In [100]:
solution = routing.SolveWithParameters(search_parameters)

I1016 15:34:50.899307 691499 search.cc:264] Start search (memory used = 1285.83 MB)
I1016 15:34:50.899423 691499 search.cc:264] Root node processed (time = 0 ms, constraints = 279, memory used = 1285.83 MB)
I1016 15:34:50.900390 691499 search.cc:264] Solution #0 (8856, time = 1 ms, branches = 41, failures = 1, depth = 33, memory used = 1285.83 MB, limit = 0%)
I1016 15:34:50.901578 691499 search.cc:264] Solution #1 (8696, objective maximum = 8856, time = 2 ms, branches = 51, failures = 3, depth = 33, Relocate<1>, neighbors = 211, filtered neighbors = 1, accepted neighbors = 1, memory used = 1285.83 MB, limit = 0%)
I1016 15:34:50.902017 691499 search.cc:264] Solution #2 (8240, objective maximum = 8856, time = 2 ms, branches = 62, failures = 6, depth = 33, Relocate<1>, neighbors = 235, filtered neighbors = 3, accepted neighbors = 2, memory used = 1285.83 MB, limit = 0%)
I1016 15:34:50.902326 691499 search.cc:264] Solution #3 (8080, objective maximum = 8856, time = 2 ms, branches = 73, fai

In [101]:
routing.status()

1

In [102]:
# store and print the results
if solution:
    print_tw_solution(data, manager, routing, solution)
    print('\n'+'='*100,'\n')
    print_load_solution(data, manager, routing, solution)

Route for vehicle 0:
0 Time(0,0) Dist(0)-> 0 Time(0,0) Dist(0)
Time of the route: 0min
Route distance: 0m

Route for vehicle 1:
0 Time(0,0) Dist(0)-> 0 Time(0,0) Dist(0)
Time of the route: 0min
Route distance: 0m

Route for vehicle 2:
0 Time(2,2) Dist(0)-> 8 Time(5,5) Dist(308)-> 6 Time(9,9) Dist(502)-> 2 Time(13,13) Dist(776)-> 0 Time(22,22) Dist(776)
Time of the route: 22min
Route distance: 776m

Route for vehicle 3:
0 Time(1,1) Dist(0)-> 7 Time(3,3) Dist(194)-> 1 Time(8,8) Dist(548)-> 4 Time(12,12) Dist(742)-> 3 Time(15,15) Dist(856)-> 0 Time(24,24) Dist(856)
Time of the route: 24min
Route distance: 856m

Route for vehicle 4:
0 Time(4,4) Dist(0)-> 9 Time(6,6) Dist(194)-> 14 Time(11,11) Dist(468)-> 16 Time(15,15) Dist(662)-> 10 Time(20,20) Dist(1016)-> 0 Time(26,26) Dist(1016)
Time of the route: 26min
Route distance: 1016m

Route for vehicle 5:
0 Time(0,0) Dist(0)-> 12 Time(4,4) Dist(388)-> 15 Time(10,10) Dist(776)-> 11 Time(14,14) Dist(1050)-> 0 Time(22,22) Dist(1050)
Time of the ro